In [1]:
import os
import multiprocessing as mp # use multiprocessing module for parallel computing
import pandas as pd # load pandas: python data analysis library
import welt_load
import numpy as np
import collections
import split_articles_welt
import itertools
import unicode_correct_welt # import the function correcting the unicode errors present in the database
import clean_welt_articles
import clean_tables_welt
from datetime import datetime

# We import some functions from the Handelsblatt folder
import sys
sys.path.insert(1, os.getcwd().replace('Welt', 'Handelsblatt'))
import count_words_mp # import the function calculating the number of words in a text
import split_number_word

# We import some functions from the SZ folder
sys.path.insert(1, os.getcwd().replace('Welt', 'SZ'))
import umlauts_correct_sz
import correct_url_sz
import count_names

# We import some functions from the 'dpa' folder
sys.path.insert(1, os.getcwd().replace('Welt', 'dpa'))
import fuzzy_duplicates_test_all_dpa
import fuzzy_duplicates_dpa # import a function that outputs the indices of fuzzy duplicates
import numeric_articles

In [2]:
NUM_CORE = mp.cpu_count()-8 # set the number of cores to use

print("The number of cores that will be used: {}".format(NUM_CORE))

The number of cores that will be used: 56


# Welt data

Welt is a distinguished national daily German newspaper that has substantial sections on both Economy and Politics. According to the IVW (Informationsgemeinschaft zur Feststellung der Verbreitung von Werbeträgern), in the first quarter of 2021, the newspaper had a circulation of 72,215 daily copies.

We downloaded the data from **LexisNexis** in July 2018, and since the project was in its early stages, we only downloaded news from the Economy and Finance sections. Our dataset includes **197,565** articles published from Monday to Saturday, between April 1999 and January 2018.

The Welt dataset is organized into a main folder named 'Welt', which contains 20 subfolders corresponding to each year from 1999 to 2018. Within each yearly subfolder, there are additional subfolders for each month, such as 'Apr 1999'. These monthly subfolders house multiple files (e.g., 'Die_Welt_1999_Apr_1.txt') that contain several articles from the respective month and year. This hierarchical organization of the Welt dataset ensures that the information is systematically stored, making it convenient for researchers to locate and extract specific articles for analysis. Unfortunately, due to copyright restrictions, we are unable to publish the data.

## Load the data

First, we need to read in the data. We create the list including the names of the 1102 files (`files_list`) and apply the function `welt_load` to them in parallel by exploiting Python's `multiprocessing` library.

We extract the following information from the txt files:

* publication date
* section/subsection of the newspaper (RUBRIK)
* the name of the newspaper
* article's title and annotation
* text of the article
* a dummy variable for the 'kompakt' section with multiple articles

In [3]:
# 'Welt' is the main folder with 20 subfolders corresponding to each year ('1999', '2000'...) in it
path = os.getcwd().replace('\\newspaper_data_processing\\Welt', '') + '\\Welt'

# Create the list of all files withing the 20 subfolders.
files_list=[]

# os.listdir(path) - names of directories: '1999', '2000'...
# we proceed folder by folder ('1999', '2000'...)
for f in [f for f in os.listdir(path) ]: 
    # again folder by folder ('Apr 1999', 'Aug 1999', ...)
    for s in [s for s in os.listdir(path + '\\' + f) ]:
        # for each file in a particular folder (e.g., Die_Welt_1999_Apr_1.txt)
        for k in [k for k in os.listdir(path +  '\\' + f + '\\' + s) ]:
            files_list.append(path + '\\' + f + '\\' + s + '\\' + k)

In [4]:
startTime = datetime.now() # track time

if __name__ == "__main__":
    pool = mp.Pool(NUM_CORE)
    data_intermediate = pool.map(welt_load.welt_load, files_list) # load data from each file in parallel
    data = pd.concat(data_intermediate) # concatenate DataFrames from different files
    pool.close()
    pool.join()
    
print(datetime.now()-startTime)

0:00:12.049826


In [5]:
data = data.sort_values(['year', 'month', 'day'], ascending=[True, True, True]) # sort the data in chronological order
data = data.reset_index() # reset the index of the DataFrame
del data['index'] # delete a column with an old index

In [6]:
data[10:15]

,year,month,day,newspaper,texts,rubrics,title,kompakt
10,1999,3,3,Welt,"[\r\n, \r\n, BANKEN\r\n, \r\n, Protestkundgebu...",Wirtschaft Kompakt,,1
11,1999,3,3,Welt,"[\r\n, \r\n, EC-KARTEN\r\n, \r\n, Neue Rufnumm...",Finanzen Kompakt,,1
12,1999,3,4,Welt,"[\r\n, \r\n, SUBVENTIONEN\r\n, \r\n, Van Miert...",Wirtschaft Kompakt,,1
13,1999,3,4,Welt,"[\r\n, \r\n, SCHIFFAHRT\r\n, \r\n, Flaute auf ...",Wirtschaft Kompakt,,1
14,1999,3,4,Welt,"[\r\n, \r\n, KRAFTWERK\r\n, \r\n, Lauchhammer ...",Wirtschaft Kompakt,,1


In [7]:
# the number of articles before pre-processing
len(data)

197565

## Split up articles

In newspaper sections where the term 'kompakt' appears, such as 'Wirtschaft kompakt', it is common to find multiple articles aggregated into a single entry. Since these individual articles may have different sentiment and address distinct topics, it is crucial to separate and process them as distinct entities.

In [8]:
mult_art = data[data.kompakt == 1]
mult_art.reset_index(inplace=True, drop=True)

In [9]:
# delete 'mult_art' from the original data
data.drop(data[data.kompakt == 1].index, inplace=True)
data.reset_index(inplace=True, drop=True)

In [10]:
# calculate chunck size 
chunk_size = int(mult_art.shape[0]/NUM_CORE)

# split data into chunks 
chunks = [mult_art.iloc[mult_art.index[i:i + chunk_size]] for 
          i in range(0, mult_art.shape[0], chunk_size)]

In [11]:
# split up articles into smaller articles and append the resulting new articles 
# to the corpus
startTime = datetime.now()

if __name__ == "__main__":
    pool = mp.Pool(NUM_CORE)
    results = pool.map(split_articles_welt.split_articles_welt, chunks) 
    pool.close()
    pool.join()
    
print(datetime.now()-startTime)

results = pd.concat(results)
print(len(results))
results.reset_index(inplace=True, drop=True) # reset the index of the DataFrame

0:00:09.248259
54410


In [12]:
# append separated articles to corpus
data = data.append(results)
data = data.sort_values(['year', 'month', 'day'], ascending=[True, True, True]) # sort the data in chronological order
data.reset_index(inplace=True, drop=True) # reset the index of the DataFrame
# the number of articles after splitting up collections of articles
print(len(data))

C:\Users\mokuneva\AppData\Local\Temp\2\ipykernel_39932\386242840.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(results)


232569


## Light pre-processing

### Remove short articles (<100 words)

The `count_words_mp` function is used to count the number of words in a text. This function only counts words and excludes numbers from the analysis. It takes into consideration Latin letters with any diacritics, such as umlauts. The function removes specific punctuation marks and non-alphabetic characters before counting the words, ensuring that the final count is accurate and relevant for sentiment analysis and topic modeling.

In [13]:
startTime = datetime.now() # track time

if __name__ == "__main__":
    pool = mp.Pool(NUM_CORE)
    count_results = pool.map(count_words_mp.count_words_mp, [text for text in data['texts']]) 
    pool.close()
    pool.join()
    
print(datetime.now()-startTime)

0:00:04.700732


In [14]:
# Save the result as a new column "word_count"
data['word_count'] = count_results

Shorter texts often lack sufficient semantic information, making it challenging for topic models and bag-of-words-based sentiment analysis tools to perform effectively. As a result, we focus on texts with more than 100 words to ensure better performance.

In [15]:
# remove articles with the number of words<100
data = data[data['word_count']>=100]
data = data.sort_values(['year', 'month', 'day'], ascending=[True, True, True]) # sort the data in chronological order
data.reset_index(inplace=True, drop=True) # reset the index of the DataFrame

In [16]:
# the number of articles after removing short articles
len(data)

176487

### Remove exact duplicates

Removing duplicates is crucial because they do not contribute any new information and only introduce noise into the analysis.

In [17]:
# All the duplicated articles are saved as 'welt_duplicates' for further exploration.
welt_duplicates = data[data['texts'].duplicated(keep = False)]

In the given corpus, we observe the following instances of duplicated articles:

1. The same article appears twice, published in different sections: e.g. in 'Finanzen kompakt' (a collection of multiple articles) and 'Finanzen'.

In [18]:
welt_duplicates.loc[[377, 378]]

,year,month,day,newspaper,texts,rubrics,title,kompakt,word_count
377,2000,11,28,Welt,GELDANLAGE. Aktienverkauf mit Verlust kann ste...,Finanzen kompakt,,1,111
378,2000,11,28,Welt,GELDANLAGE. Aktienverkauf mit Verlust kann ste...,Finanzen,,1,111


2. The same article (here, an overview of a series) is included in the corpus with different publication dates (here, 8.04.2014 and 9.04.2014). In such cases, a logical approach is to retain the first entry.

In [19]:
welt_duplicates.loc[[152567, 152587]]

,year,month,day,newspaper,texts,rubrics,title,kompakt,word_count
152567,2014,4,8,Welt,Serie im Überblick. Teil 1: Lohn und Arbeitgeb...,FINANZEN,Serie im Überblick.,0,112
152587,2014,4,9,Welt,Serie im Überblick. Teil 1: Lohn und Arbeitgeb...,FINANZEN,Serie im Überblick.,0,112


In [20]:
# drop the exact duplicates, keep the article with the earlier publication date ('first')
data.drop_duplicates(['texts'], keep = 'first', inplace=True)
data.reset_index(inplace=True, drop=True)

In [21]:
# the number of articles after removing exact duplicates
len(data)

175482

## Filtering

### Section

Articles from the economic and financial sections of Welt are categorized into 50 distinct subsections. To maintain consistency in topical content over time, we exclude articles from certain subsections that were only covered during a limited time period.

In [22]:
# Subsections and the number of articles per subsection
counter=collections.Counter(data['rubrics'])
print(counter.most_common(1000))

[('WIRTSCHAFT', 66549), ('Wirtschaft', 53340), ('FINANZEN', 27164), ('Finanzen', 20572), ('Hamburg Wirtschaft', 6132), ('Berlin Wirtschaft', 971), ('Wirtschaft Kompakt', 269), ('Wirtschaft kompakt', 105), ('Finanzen kompakt', 98), ('Berliner Web Wirtschaft', 25), ('Extra', 24), ('FINANZEN Ausg. 241', 24), ('HH-WIRTSCHAFT', 20), ('FINANZEN Ausg. 228', 19), ('Extra Messe-Wirtschaft', 18), ('FINANZEN Ausg. 105', 16), ('Finanzen Kompakt', 15), ('KLIMA & FINANZEN Ausg. 19', 15), ('FINANZEN Ausg. 115', 11), ('Berliner Wirtschaft', 10), ('Wissenschaft', 8), ('FINANZEN Ausg. 90', 8), ('Berlin Werbe Wirtschaft', 6), ('BERLIN WIRTSCHAFT', 6), ('finanzen kompakt', 6), ('WIRTSCHAFT GESAMT', 6), ('Hamburg', 5), ('WebWelt', 4), ('wirtschaft-bestseller', 3), ('Sonderseite Messe-Wirtschaft', 3), ('FINANZEN Ausg. 76', 3), ('WebWirtschaft', 2), ('Investment-Tipps', 2), ('Berlin', 2), ('Wirtschaft - Termine', 2), ('HAMBURG', 2), ('WISSENSCHAFT', 2), ('TITEL', 2), ('Sonderseite Messe Wirtschaft', 2), ('Wi

The `keep_sections` variable represents a list of subsections that are retained for subsequent analysis.

In [23]:
keep_sections = ['WIRTSCHAFT', 'Wirtschaft', 'FINANZEN', 'Finanzen', 'Wirtschaft kompakt', 'Wirtschaft Kompakt', 
                 'Finanzen kompakt', 'Finanzen Kompakt', 'wirtschaft kompakt', 'finanzen kompakt']

In [24]:
# Keep only relevant sections
data = data[data['rubrics'].isin(keep_sections)]
data.reset_index(inplace=True, drop=True)

In [25]:
# the number of articles after removing articles from subsections with limited time coverage
len(data)

168118

### Title

Exclude articles with the following title patterns:

* 'Verlierer.': stock prices only.
* 'Gewinner.': stock prices only.
* 'Auf dem Sprung an die Börse.': quantitative info on companies planning to go public.
* 'Aktuelle Neuemissionen.': quantitative info on new issues.
* 'Zinsen in Prozent.': quantitative info on interest rates. 
* 'Die Schlagzeile vom 1962.' ('Die Schlagzeile vom 1963.'): headlines from 1962/1963, historical news.
* 'Leserfrage der Woche.' ('Leserfrage.'): reader's questions unrelated to the economy, typically legal questions.
* 'Messe-Termine.': trade show dates.
* 'service splitter.' ('ServiceSplitter.', 'Service-Splitter.', 'Service Splitter.') : unrelated to the economy.
* 'Die Steuer-Serie auf einen Blick.': tax series, not economy-related.
* 'WebWegWeiser.': web news.
* 'Die Titel mit den größten Kursausschlägen. Stand: Schluss.': stock prices.
* 'Wirtschaft. Vernetzt im Wohnzimmer.' ('Vernetzt im Wohnzimmer.'): advertisement of digital products.
* 'Telefonaktion.': advertisement for phone-based Q&A on specific topics.
* 'TICKER.' ('Ticker.'): stock prices.
* 'Heizöl-Preise aktuell.': current heating oil prices.
* 'Die Titel mit den größten Kursausschlägen. Schluss.': stock prices.
* 'Telefontarife.': phone rates (numbers).
* 'heute im sportteil.': today in sports section (irrelevant).
* 'WELT-Rangliste der deutschen Wirtschaft - Auf einen Blick: Umsatz und. Beschäftigung; Die 500 größten Unternehmen des Landes auf einen Blick: Umsatz, Branchen und Beschäftigte.': sales and employment data for the largest companies (numbers).
* 'Serie im Überblick.' ('Die Serie im Überblick.'): short introduction to the series.
* 'So können Sie Ihr Portfolio optimieren.': the asset check organized by Welt.
* 'Richtig vorsorgen. So finden Sie den perfekten Riester-Vertrag.': finding the perfect pension plan.
* 'Die Titel mit den größten Kursausschlägen. Stand: 16.00 Uhr.': stock prices.
* 'wirtschaft-bestseller.' ('Wirtschaft-Bestseller.'): book prices.
* 'Themen (Fotos).': introduction to the topics discussed in the issue.
* 'Emittenten-Kontakte.': contact information for warrant issuers.
* 'TERMINE': a list of upcoming events.
* 'Kursteil erhält ein neues Gesicht': an announcement about an update to the stock market section.
* 'Wirtschaft und Politik.': schedule of important events.
* '^Termine vom': schedule of important events.
* 'Wirtschaftstermine.': schedule of important events.

In [26]:
title_clean = ['Verlierer.', 'Gewinner.', 'Auf dem Sprung an die Börse.', 'Aktuelle Neuemissionen.',
              'Zinsen in Prozent.', 'Die Schlagzeile vom 1962.', 'Leserfrage der Woche.',
              'Messe-Termine.', 'service splitter.', 'ServiceSplitter.', 'Service-Splitter.', 'Service Splitter.',
              'Die Steuer-Serie auf einen Blick.', 'WebWegWeiser.',
              'Die Titel mit den größten Kursausschlägen. Stand: Schluss.',
              'Wirtschaft. Vernetzt im Wohnzimmer.', 'Telefonaktion.', 'TICKER.',
              'Vernetzt im Wohnzimmer.', 'Die Schlagzeile vom 1963.', 'Heizöl-Preise aktuell.',
              'Die Titel mit den größten Kursausschlägen. Schluss.', 'Telefontarife.',
              'heute im sportteil.', 
              'WELT-Rangliste der deutschen Wirtschaft - Auf einen Blick: Umsatz und. Beschäftigung; Die 500 größten Unternehmen des Landes auf einen Blick: Umsatz, Branchen und Beschäftigte.',
              'Serie im Überblick.', 'So können Sie Ihr Portfolio optimieren.',
              'Richtig vorsorgen. So finden Sie den perfekten Riester-Vertrag.',
              'Die Serie im Überblick.', 'Leserfrage.', 'Ticker.',
              'Die Titel mit den größten Kursausschlägen. Stand: 16.00 Uhr.'
              'wirtschaft-bestseller.', 'Wirtschaft-Bestseller.', 'Themen (Fotos).',
              'Emittenten-Kontakte.',
              'Wirtschaft und Politik.',
              'Wirtschaftstermine.']

In [27]:
data = data[~data['title'].isin(title_clean)]
data.reset_index(inplace=True, drop=True) # reset the index of the DataFrame

In [28]:
title_patterns = ['TERMINE', 'Kursteil erhält ein neues Gesicht', '^Termine vom']

In [29]:
data = data[~data['title'].str.contains('|'.join(title_patterns))]
data.reset_index(inplace=True, drop=True) # reset the index of the DataFrame

In [30]:
# the number of articles after excluding articles based on the title patterns
len(data)

167735

## Umlauts

In the given dataset, some texts contain incorrect umlaut encodings, such as '&auml;', '&uuml;', '&ouml;', '&Auml;', '&Uuml;', and '&Ouml;'. These encodings are typically used in HTML documents to represent umlaut characters like 'ä', 'ü', 'ö', 'Ä', 'Ü', and 'Ö'. However, in this case, the encodings are not being interpreted correctly, leading to broken umlauts in the text.

To fix this issue, we use the function `umlauts_correct_sz` that replaces these incorrect umlaut encodings with the correct umlaut characters.

In [31]:
# Define a list of incorrect umlaut encodings
incorrect_umlauts = ["&auml;", "&uuml;", "&ouml;", "&Auml;", "&Uuml;", "&Ouml;"]

pattern = "|".join(incorrect_umlauts)
welt_uml_encoding = data[data.texts.str.contains(pattern)]

welt_uml_encoding.iloc[0]['texts']

'Britische Versicherer leiden unter Börsenflaute. Dividenden werden gekürzt - Finanzaufsicht soll die Anlagevorschriften lockern. London - Britische Lebensversicherer galten viele Jahre als Geheimtipp. Während deutsche Assekuranzen wegen der strengen Anlagevorschriften nur einen geringen Teil der Prämien in Aktien investieren durften und deshalb mit mündelsicheren Anleihen und Pfandbriefen nur niedrige einstellige Renditen erwirtschafteten, konnten britische Konkurrenten bis zu 80 Prozent der Spargelder in Aktien investieren. So lange die Kurse an den Börsen stiegen, waren hohe Gewinne garantiert. Doch seit die Finanzmärkte die schlimmste Baisse seit dem zweiten Weltkrieg erleben, haben die britischen Lebensversicherer ein Problem. Mit fallenden Kursen sinken automatisch die Renditen. Gleichzeitig müssen die Versicherungspolicen bedient werden. Um flüssig zu bleiben, sind die Assekuranzen gezwungen, Aktien zu verkaufen. Das wiederum setzt die Kurse zusätzlich unter Druck. Britannic mus

In [32]:
startTime = datetime.now() # track time

if __name__ == "__main__":
    pool = mp.Pool(NUM_CORE)
    texts_corrected = pool.map(umlauts_correct_sz.umlauts_correct_sz, [text for text in data['texts']]) 
    pool.close()
    pool.join()
    
print(datetime.now()-startTime)

0:00:03.434115


In [33]:
data['texts'] = texts_corrected

## Remove URLs

A function called `correct_url_sz` takes a text input and processes it to remove URLs, HTML file references, and email addresses. A set called `exceptions` contains specific URLs that are not removed from the text. Preserving specific URLs, such as 'amazon.de', is crucial for maintaining the context and sentence structure, as removing them could result in loss of important information or topic identification, especially when certain internet companies are known primarily by their website names.

In [34]:
startTime = datetime.now()

if __name__ == "__main__":
    pool = mp.Pool(NUM_CORE)
    url_corrected = pool.map(correct_url_sz.correct_url_sz, [text for text in data['texts']]) 
    pool.close()
    pool.join()

print(datetime.now()-startTime)

0:00:13.657538


In [35]:
data['texts'] = url_corrected

## Fixing tokens containing a number and a word

We have identified several cases where numbers and words are erroneously merged into a single token. To address this problem of merged tokens, the following code has been implemented.

In [36]:
startTime = datetime.now()

if __name__ == "__main__":
    pool = mp.Pool(NUM_CORE)
    split_corrected = pool.map(split_number_word.split_number_word, [text for text in data['texts']]) 
    pool.close()
    pool.join()

print(datetime.now()-startTime)

0:00:04.893910


In [37]:
data['texts'] = split_corrected

## Remove fuzzy duplicates

By fuzzy duplicates we understand nearly duplicated articles. These are:

* drafts/minor revisions of the articles saved in the database;
* slightly changed advertisements which are published several times during a month.

We identify 'fuzzy' duplicates using cosine similarity and choose a threshold of 93% based on some visual exploration. Here is the article by Ryan Basques we used as a reference: [Link](https://towardsdatascience.com/a-laymans-guide-to-fuzzy-document-deduplication-a3b3cf9a05a7). 

In [38]:
# Create a 'date' column in the data DataFrame
data['date'] = pd.to_datetime(data[['year', 'month', 'day']])

### Duplicates exploration

We must first identify and visually inspect the fuzzy duplicates using the `fuzzy_duplicates_test` function before proceeding with their removal. This step ensures the algorithm's performance is as expected, and it accurately identifies articles as fuzzy duplicates based on their cosine similarity.

In [39]:
# Required input for the function 'fuzzy_duplicates': a dataframe for each month-year combination.
# List with a year
inputs_year = []
# List with a month
inputs_month = []
# List with the dataframes containing 'year', 'month', and 'texts' columns
inputs_month_year = []
for year in list(set(data['year'])):
    for month in list(set(data['month'])):
        # Exclude time periods with no data
        if data[(data['year'] == year) & (data['month'] == month)].empty:
            continue
        inputs_year.append(year)
        inputs_month.append(month)
        inputs_month_year.append(data[(data['year'] == year) & (data['month'] == month)][["month", "year", "texts", "word_count", "date"]])
        
inputs = list(zip(inputs_year, inputs_month, inputs_month_year))

In [40]:
startTime = datetime.now() # track time

import fuzzy_duplicates_test_all_dpa 

if __name__ == "__main__":
    pool = mp.Pool(NUM_CORE)
    dup_intermediate = pool.map(fuzzy_duplicates_test_all_dpa.fuzzy_duplicates_test, inputs) 
    duplicates = pd.concat(dup_intermediate) 
    pool.close()
    pool.join()
    
print(datetime.now()-startTime)

duplicates.to_csv('duplicates.csv', encoding = 'utf-8-sig', sep = ';')

0:03:10.864753


### Drop the duplicates

In [41]:
startTime = datetime.now() # track time

delete_indices = []

if __name__ == "__main__":
    pool = mp.Pool(NUM_CORE)
    # apply function to all combinations of month-year in parallel
    delete_intermediate = pool.map(fuzzy_duplicates_dpa.fuzzy_duplicates, inputs)
    delete_indices = delete_indices + delete_intermediate # create one list of indices
    pool.close()
    pool.join()
    
print(datetime.now()-startTime)

0:03:34.546829


In [42]:
# Free memory
inputs = None
# List of indices corresponding to the duplicated articles
delete_indices = [item for sublist in delete_indices for item in sublist]
# List of unique indices
delete_indices = list(set(delete_indices))
# Drop the fuzzy duplicates
data.drop(data.index[delete_indices], inplace = True)
data.reset_index(inplace=True, drop=True) # reset the index of the DataFrame

In [43]:
# the number of articles after removing fuzzy duplicates
len(data)

166603

## Welt-specific problem

We encountered a minor issue specific to the Welt dataset while addressing other problems. This issue involves texts containing unicode errors. To resolve this, we identify such texts and correct the unicode errors using the function `unicode_correct_welt`. For example, the function converts 'Ha{ring}kann' to 'Håkan'.

In [44]:
startTime = datetime.now() 

to_correct = list(data[data.texts.str.contains('{sbquo}|{ring}|{cech}|{ogon}|{circ}')].index)

if __name__ == "__main__":
    pool = mp.Pool(NUM_CORE)
    unicode_corrected = pool.map(unicode_correct_welt.unicode_correct_welt, [text for text in data.loc[to_correct, 'texts']]) 
    pool.close()
    pool.join()
    
print(datetime.now()-startTime)

# Replace the texts with the corrected version of the articles.
data.loc[to_correct, 'texts'] = unicode_corrected

0:00:00.817437


## Clean articles

Welt articles include some text passages that are unlikely to be relevant for either topic modeling or sentiment analysis. We decided to clean the affected articles from these text passages to make the analysis easier for our models.

We remove the following information from the texts:

   * 1) Internet addresses
   * 2) references to photo sources
   * 3) references to additional information
   * 4) telephone and fax numbers
   * 5) references to page numbers
   * 6) advertisements for the new Gabler-Wirtschaftslexikon

In [45]:
startTime = datetime.now()

if __name__ == "__main__":
    pool = mp.Pool(NUM_CORE)
    cleaned_articles = pool.map(clean_welt_articles.clean_welt_articles, [text for text in data['texts']]) 
    pool.close()
    pool.join()
    
print(datetime.now()-startTime)

0:00:10.341506


In [46]:
data['texts'] = cleaned_articles

In [47]:
# Calculate an updated word count of the cleaned articles.

startTime = datetime.now()

if __name__ == "__main__":
    pool = mp.Pool(NUM_CORE)
    count_results = pool.map(count_words_mp.count_words_mp, [text for text in data['texts']]) 
    pool.close()
    pool.join()
    
print(datetime.now()-startTime)

0:00:03.923680


In [48]:
# Update the word count in the data frame.
data['word_count'] = count_results
# Drop short articles.
data = data[data['word_count']>=100]
data.reset_index(inplace=True, drop=True)

In [49]:
# the number of articles after excluding articles shorter than 100 words
len(data)

166552

## Exclude tables

In this section, we preprocess the news articles by removing tables to minimize noise and emphasize relevant content. We begin by calculating a numerical density metric for each text, which is computed as the ratio of the count of numbers to the total word count (excluding numbers). Texts with a numerical density of at least 20% are considered as candidates for containing tables. We manually examine these texts to identify recurring strings that typically precede tables. Using regular expressions, we exclude the tables based on these strings. Moreover, we delete some text segments predominantly comprising numbers.

In [50]:
# use the 'numeric_articles' function to identify articles with a high share of numbers in them
inputs = zip(data['texts'], data['word_count'], itertools.repeat(0.20))

startTime = datetime.now()

if __name__ == "__main__":
    pool = mp.Pool(NUM_CORE)
    tables = pool.starmap(numeric_articles.numeric_articles, inputs)
    pool.close()
    pool.join()

print(datetime.now()-startTime)
data['tables'] = tables
tables = data[data.tables == True]

0:00:02.793106


Below is an example of an article containing a table with the losses of the ten richest Americans. To remove this table, we apply the following regular expression: `r'Tabelle: Die Verluste.{0,}'`.

In [51]:
tables.loc[105975,'texts']

'Gigantische Übernahme. US-Milliardär Buffett kauft sich eine Eisenbahn. HIGHLIGHT: Der US-Investor Warren Buffett ist geschätzte 40 Milliarden Dollar reich. Nun kauft sich der Milliardär eine komplette Eisenbahn-Gesellschaft - und er gibt dafür noch mehr Geld aus, als ihm als Vermögen zugeschrieben wird. Buffett sagt: Das ist wie eine Wette - und setzt nun alles auf eine Karte. Die Holdinggesellschaft Berkshire Hathaway des Milliardärs Warren Buffett übernimmt für 44 Milliarden US-Dollar - umgerechnet rund 30 Milliarden Euro - die Bahngesellschaft Burlington Northern Santa Fe in den Vereinigten Staaten von Amerika. Damit stemmt Buffett den größten Zukauf in der Geschichte seines Unternehmens. Er wird in der Forbes-Liste der reichsten Menschen 2009 mit 40 Milliarden Dollar Vermögen geführt. Der Zukauf übersteigt somit sogar Buffetts Reichtum. Er hatte in der Finanzkrise mehrere Milliarden Dollar verloren. Berkshire zahle 100 Dollar je Burlington-Aktie in bar und in Dividendenpapieren, 

Clean tables using regular expressions.

In [52]:
startTime = datetime.now()
if __name__ == "__main__":
    pool = mp.Pool(NUM_CORE)
    remove_tables = pool.map(clean_tables_welt.clean_tables_welt, [text for text in data['texts']]) 
    pool.close()
    pool.join()
    
print(datetime.now()-startTime)

data['texts'] = remove_tables

0:00:04.014743


In [53]:
startTime = datetime.now()

if __name__ == "__main__":
    pool = mp.Pool(NUM_CORE)
    count_results = pool.map(count_words_mp.count_words_mp, [text for text in data['texts']]) 
    pool.close()
    pool.join()
    
print(datetime.now()-startTime)

0:00:04.017384


In [54]:
# Update the content of the column "word_count"
data['word_count'] = count_results

In [55]:
# remove articles with less than 100 words
data = data[data['word_count']>=100]
data = data.sort_values(['year', 'month', 'day'], ascending=[True, True, True]) # sort the data in chronological order
data.reset_index(inplace=True, drop=True)
# the number of articles after excluding articles shorter than 100 words
print(len(data))
del data['tables']

166541


## Identify articles that predominantly consist of numbers

Articles that predominantly consist of numerical data present challenges for sentiment or topic analysis as, once numbers are removed, they tend to contain limited information. Therefore, we remove articles with a numerical density of 50% or greater.

In [56]:
# use the 'numeric_articles' function to identify economic articles with a high share of numbers in them
inputs = zip(data['texts'], data['word_count'], itertools.repeat(0.50))

startTime = datetime.now()

if __name__ == "__main__":
    pool = mp.Pool(NUM_CORE)
    numeric_list = pool.starmap(numeric_articles.numeric_articles, inputs)
    pool.close()
    pool.join()

print(datetime.now()-startTime)
data['numeric'] = numeric_list

0:00:02.955905


In [57]:
# inspect example article with high share of numbers
data[data.numeric == True]['texts'].iloc[0]

'Bundesliga Hamburg - Wolfsburg...Sa., 15.30 Schalke - Bayern München...Sa., 15.30 Kaiserslautern - Dortmund...Sa., 15.30 Stuttgart - Mönchengladbach...Sa., 15.30 Mainz - Leverkusen...Sa., 15.30 Bielefeld - Hertha...Sa., 15.30 Bremen - Nürnberg...Sa., 15.30 Köln - Hannover...So., 17.30 Duisburg - Frankfurt...So., 17.30 1. FC Bayern München 8 17:5 21 2. Hamburger SV 8 17:4 20 3. Werder Bremen 8 21:10 19 4. FC Schalke 04 8 10:5 16 5. Hertha BSC Berlin 8 12:9 14 6. Borussia Mönchengladbach 8 10:9 14 7. VfL Wolfsburg 8 11:12 12 8. Bayer 04 Leverkusen 8 15:14 11 9. Borussia Dortmund 8 11:10 10 10. VfB Stuttgart 8 9:9 10 11. 1. FC Köln 8 11:13 9 12. Hannover 96 8 9:12 7 13. 1. FC Kaiserslautern 8 10:18 7 14. MSV Duisburg 8 11:16 6 15. 1. FC Nürnberg 8 6:11 6 16. Arminia Bielefeld 8 6:13 6 17. 1. FSV Mainz 05 8 5:12 4 18. Eintr. Frankfurt 8 3:12 4 BASKETBALL Bundesliga, Männer: EWE Bask. Oldenburg - TBB Trier 90:73. EISHOCKEY DEL, 11. Spieltag: Augsburger Panther - Eisbären Berlin 5:4 n.P. FE

In [58]:
data[data.numeric == True]['texts'].iloc[1]

"Deutschlands Große 500. Die WELT-Rangliste der deutschen Wirtschaft (Teil I, Rang 1-276). RangUnternehmenBrancheUmsatzAnmerkungenBeschäftigte. 09082009+/-%2009+/-%. 1 1 Volkswagen AG, Wolfsburg Automobil 105.187 - 7,6 K, IAS/IFRS 368.500 - 0,4 2 3 Eon AG, Düsseldorf Energie 81.817 - 5,7 K 88.586 - 5,6 3 2 Daimler AG, Stuttgart Automobil 78.924 - 19,8 K, IFRS 256.407 - 6,2 4 4 Siemens AG, München Elektronik, Elektrotechnik 76.651 - 0,9 K, IFRS, 405.000 - 5,2. 5 5 Metro AG, Düsseldorf Handel 65.529 - 3,6 K, IAS/ IFRS 286.091 - 1,7 6 7 Deutsche Telekom AG, Bonn (*1) Telekommunikation 64.600 + 4,8 K, IAS/IFRS, ÄK 259.920 + 14,1. 7 - Porsche Automobil Holding SE, Stuttgart (*2) Automobilindustrie 57.081 - K, IAS/IFRS 375.959 -. 8 9 Unternehmensgruppe Schwarz, Neckarsulm Lebensmitteleinzelhandel 54.500 + 0,9 K, G1 290.000 + 3,6. 9 12 Rewe Group, Köln Handel, Touristik 50.900 + 2,2 K, IAS/IFRS, G1 326.000 + 2,1. 10 6 BASF S.E., Ludwigshafen Chemie 50.693 - 18,6 K, IAS/IFRS 104.779 + 8,1 11 1

In [59]:
len(data[data.numeric == True])

7

In [60]:
# drop articles predominantly consisting of numbers
data = data[data.numeric == False]
del data['numeric']
data.reset_index(inplace=True, drop=True)
# the number of articles after removing articles that predominantly consist of numbers
print(len(data))

166534


## Identify articles that predominantly consist of names

We eliminate texts with a name density of at least 15% (relative to the total word count, excluding numbers) as part of our pre-processing pipeline. This exclusion is important to guarantee that the remaining articles contain sufficient content for effective topic analysis, as the removal of common German names is a standard pre-processing step in LDA model estimation.

In [61]:
path = os.getcwd().replace('\\Welt', '') + '\\SZ'

# Load the dictionary containing common German first and last names
with open(path + "\\names.txt", "r", encoding="utf-8-sig") as f:
    names_list = f.read().splitlines()

In [62]:
inputs = zip(data['texts'], data['word_count'], itertools.repeat(names_list))

startTime = datetime.now()

if __name__ == "__main__":
    pool = mp.Pool(NUM_CORE)
    names_result = pool.starmap(count_names.count_names, inputs)
    pool.close()
    pool.join()

print(datetime.now()-startTime)

data['names'] = names_result

0:00:21.498105


In [63]:
# Example of an article with a high proportion of names
data[data.names>=0.15].iloc[0]['texts']

'Die Mitglieder der Kommission. Der Kommission unter der Leitung des Ökonomen Bert Rürup gehören 18 Männer und acht Frauen an. Aus der Politik: Heinrich Tiemann, Staatssekretär im Gesundheits- und Sozialministerium, Rosemarie Wilcken, Wismarer SPD-Bürgermeisterin und Gitta Trauernicht, niedersächsische. SPD-Sozialministerin. Aus der Wissenschaft: die Ökonomen Axel Börsch-Supan, Bernd Raffelhüschen und Gert Wagner, die Verwaltungswissenschaftlerin Gisela Färber, die Gesundheitswissenschaftler Karl W. Lauterbach und Eckhard Nagel, die Politikprofessoren Frank Nullmeier und Josef Schmid. Aus der Wirtschaft: Unternehmensberater Roland Berger, Claus-Michael Dill, Vorstandschef des AXA-Konzerns, die Unternehmerin Dominique Döttling, Günther Fleig, Vorstand bei Daimler-Chrysler, und BASF-Vorstand Eggert Voscherau. Jürgen Husmann vertritt die Arbeitgeberverbände (BDA) und ist Vorstand der Bundesversicherungsanstalt für Angestellte (BfA). Von den Gewerkschaften: Vize-Chefin des DGB, Ursula Enge

In [64]:
# Exclude texts with a name density of at least 15%
data = data[data.names<0.15]
del data['names']
data.reset_index(inplace=True, drop=True)
# the number of articles after removing articles that predominantly consist of names
print(len(data))

166510


In [65]:
counter=collections.Counter(data['year'])
print(counter)

Counter({2003: 13950, 2005: 13615, 2006: 13381, 2007: 12874, 2009: 12171, 2002: 11564, 2008: 11487, 2001: 9932, 2010: 9830, 2004: 8419, 2013: 7651, 2011: 7498, 2012: 7223, 2014: 6990, 2015: 6473, 2016: 6216, 2017: 5603, 2000: 899, 2018: 460, 1999: 274})


In [66]:
data.to_csv('welt_prepro_final.csv', encoding = 'utf-8-sig', sep = ';')